In [1]:
# packages
import pymysql.cursors
import pandas as pd
import os

In [2]:
os.listdir()

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'Data',
 'Data Warehouse',
 'etl.py',
 'modelagem_dimensional.ipynb']

In [3]:
# reading just 100000 rows the dataframe 
df = pd.read_csv('Data/201701_BolsaFamiliaFolhaPagamento.csv', encoding = "ISO-8859-1", error_bad_lines=False, sep='\t', nrows=1000)

In [4]:
df.shape

(1000, 12)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
UF                        1000 non-null object
Código SIAFI Município    1000 non-null int64
Nome Município            1000 non-null object
Código Função             1000 non-null int64
Código Subfunção          1000 non-null int64
Código Programa           1000 non-null int64
Código Ação               1000 non-null int64
NIS Favorecido            1000 non-null int64
Nome Favorecido           1000 non-null object
Fonte-Finalidade          1000 non-null object
Valor Parcela             1000 non-null float64
Mês Competência           1000 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 93.8+ KB


In [70]:
df.head(10)

,UF,Código SIAFI Município,Nome Município,Código Função,Código Subfunção,Código Programa,Código Ação,NIS Favorecido,Nome Favorecido,Fonte-Finalidade,Valor Parcela,Mês Competência
0,BA,3389,BROTAS DE MACAUBAS,8,244,1335,8442,16074176737,FLAVIANO SEBASTIAO DOS SANTOS,CAIXA - Programa Bolsa Família,131.00,01/2017
1,PE,2531,RECIFE,8,244,1335,8442,20033347012,ELANE PATRICIA DA SILVA DAMASIO NUNES,CAIXA - Programa Bolsa Família,170.00,01/2017
2,SP,6291,CAMPINAS,8,244,1335,8442,13287063897,ANGELA APARECIDA SERAFIM,CAIXA - Programa Bolsa Família,202.00,01/2017
3,PI,1167,PIRIPIRI,8,244,1335,8442,16356178516,LUIZA FLORINDA DA SILVA,CAIXA - Programa Bolsa Família,85.00,01/2017
4,MA,755,CARUTAPERA,8,244,1335,8442,16099396730,EDNA MARIA FERREIRA PINHEIRO,CAIXA - Programa Bolsa Família,85.00,01/2017
5,PE,2389,CHA GRANDE,8,244,1335,8442,16027702517,ANA MARIA DA SILVA,CAIXA - Programa Bolsa Família,124.00,01/2017
6,BA,3765,OLIVEIRA DOS BREJINHOS,8,244,1335,8442,23672637749,IVANILSON DOS SANTOS MACHADO,CAIXA - Programa Bolsa Família,85.00,01/2017
7,CE,1585,MARACANAU,8,244,1335,8442,16390347554,MARIA DO SOCORRO ALVES DA SILVA,CAIXA - Programa Bolsa Família,124.00,01/2017
8,SE,3191,NOSSA SENHORA DAS DORES,8,244,1335,8442,20947248433,MARIA JOSINETE DOS SANTOS,CAIXA - Programa Bolsa Família,163.00,01/2017
9,BA,3495,CRUZ DAS ALMAS,8,244,1335,8442,12461489000,JOSEILZA SOUZA,CAIXA - Programa Bolsa Família,39.00,01/2017


In [7]:
for col in df.columns:
    print("Coluna: ", col)

Coluna:  UF
Coluna:  Código SIAFI Município
Coluna:  Nome Município
Coluna:  Código Função
Coluna:  Código Subfunção
Coluna:  Código Programa
Coluna:  Código Ação
Coluna:  NIS Favorecido
Coluna:  Nome Favorecido
Coluna:  Fonte-Finalidade
Coluna:  Valor Parcela
Coluna:  Mês Competência


In [8]:
df['Regiao'] = None

In [9]:
# add region name to dataframe 
def regiao(estado):
    if estado in ['MA','PI','CE','RN','PE','PB','SE','AL','BA']:
        return 'Nordeste'
    elif estado in ['AM', 'RR', 'RO', 'AP', 'TO', 'RO', 'AC']:
        return 'Norte'
    elif estado in ['MT', 'MS', 'GO', 'DF']:
        return 'Centro-Oeste'
    elif estado in ['SP', 'RJ', 'ES', 'MG']:
        return 'Sudeste'
    else:
        return 'Sul'

In [10]:
df['Regiao'] = df['UF'].apply(regiao)

In [11]:
df.columns

Index(['UF', 'Código SIAFI Município', 'Nome Município', 'Código Função',
       'Código Subfunção', 'Código Programa', 'Código Ação', 'NIS Favorecido',
       'Nome Favorecido', 'Fonte-Finalidade', 'Valor Parcela',
       'Mês Competência', 'Regiao'],
      dtype='object')

In [14]:
# returning state's name to datafrane
def estado(uf):
    if uf == 'MA':
        return 'Maranhão'
    elif uf == 'PI':
        return 'Piauí'
    elif uf == 'CE':
        return 'Ceará'
    elif uf == 'BA':
        return 'Bahia'
    elif uf == 'AL':
        return 'Alagoas'
    elif uf == 'SE':
        return 'Sergipe'
    elif uf == 'RN':
        return 'Rio Grande do Norte'
    elif uf == 'PE':
        return 'Pernambuco'
    elif uf == 'PB':
        return 'Paraíba'
    elif uf == 'AM':
        return 'Amazonas'
    elif uf == 'RR':
        return 'Roraima'
    elif uf == 'AP':
        return 'Amapá'
    elif uf == 'PA':
        return 'Pará'
    elif uf == 'TO':
        return 'Tocantins'
    elif uf == 'RO':
        return 'Rondonia'
    elif uf == 'AC':
        return 'Acre'
    elif uf == 'MT':
        return 'Mato Grosso'
    elif uf == 'MS':
        return 'Mato Grosso do Sul'
    elif uf == 'GO':
        return 'Goiás'
    elif uf == 'SP':
        return 'São Paulo'
    elif uf == 'RJ':
        return 'Rio de Janeiro'
    elif uf == 'ES':
        return 'Espirito Santo'
    elif uf == 'MG':
        return 'Minas Gerais'
    elif uf == 'PR':
        return 'Paraná'
    elif uf == 'RS':
        return 'Rio Grande do Sul'
    elif uf == 'SC':
        return 'Santa Catarina'
    else:
        return ''

In [15]:
#df['Estado'] = None
df['Estado'] = df['UF'].apply(estado)

In [16]:
df['Fonte'] = df['Fonte-Finalidade'].apply(lambda x: x.split('-')[0])
df['Finalidade'] = df['Fonte-Finalidade'].apply(lambda x: x.split('-')[1])
df.drop('Fonte-Finalidade', axis=1, inplace = True)

In [18]:
df.head(3)

,UF,Código SIAFI Município,Nome Município,Código Função,Código Subfunção,Código Programa,Código Ação,NIS Favorecido,Nome Favorecido,Valor Parcela,Mês Competência,Regiao,Estado,Fonte,Finalidade
0,BA,3389,BROTAS DE MACAUBAS,8,244,1335,8442,16074176737,FLAVIANO SEBASTIAO DOS SANTOS,131.0,01/2017,Nordeste,Bahia,CAIXA,Programa Bolsa Família
1,PE,2531,RECIFE,8,244,1335,8442,20033347012,ELANE PATRICIA DA SILVA DAMASIO NUNES,170.0,01/2017,Nordeste,Pernambuco,CAIXA,Programa Bolsa Família
2,SP,6291,CAMPINAS,8,244,1335,8442,13287063897,ANGELA APARECIDA SERAFIM,202.0,01/2017,Sudeste,São Paulo,CAIXA,Programa Bolsa Família


In [19]:
df['Estado'].unique()
df['Regiao'].unique()

array(['Nordeste', 'Sudeste', 'Centro-Oeste', 'Sul', 'Norte'],
      dtype=object)

In [20]:
df[df['Nome Município'] == 'RECIFE']['Código SIAFI Município'].unique()[0]

2531

In [22]:
df.isnull().any()

UF                        False
Código SIAFI Município    False
Nome Município            False
Código Função             False
Código Subfunção          False
Código Programa           False
Código Ação               False
NIS Favorecido            False
Nome Favorecido           False
Valor Parcela             False
Mês Competência           False
Regiao                    False
Estado                    False
Fonte                     False
Finalidade                False
dtype: bool

In [24]:
df.columns

Index(['UF', 'Código SIAFI Município', 'Nome Município', 'Código Função',
       'Código Subfunção', 'Código Programa', 'Código Ação', 'NIS Favorecido',
       'Nome Favorecido', 'Valor Parcela', 'Mês Competência', 'Regiao',
       'Estado', 'Fonte', 'Finalidade'],
      dtype='object')

In [47]:
# loading data to database
def load_data(dataset):
    # Connect to the database
    connection = pymysql.connect(host='localhost', user='root', password='', db='BolsaFamilaDW', charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)

    SQL = ''
    try:
        with connection.cursor() as cursor:
            # Adding data to DimMunicipio
            for cidade in dataset['Nome Município'].unique():
                # Create a new record
                dataset_aux = dataset[dataset['Nome Município'] == cidade]

                cod_siafi = dataset_aux['Código SIAFI Município'].unique()[0]
                estado = dataset_aux['Estado'].unique()[0]
                regiao = dataset_aux['Regiao'].unique()[0]
                uf = dataset_aux['UF'].unique()[0]
                sql = """INSERT INTO DimMunicipio (COD_SIAFI, MUNICIPIO, ESTADO, REGIAO, UF) VALUES (%d, "{}", "{}", "{}", "{}")""".format(int(cod_siafi), cidade, estado, regiao, uf)
                cursor.execute(sql)#, ('webmaster@python.org', 'very-secret'))

            # adding data to DimFavorecido
            for nome in dataset['Nome Favorecido'].unique():
                dataset_aux = dataset[dataset['Nome Favorecido'] == nome]
                
                nis = dataset_aux['NIS Favorecido'].unique()[0]
                sql = """INSERT INTO DimFavorecido(NIS, NOME) VALUES ("{}", "{}")""".format(nis, nome)
                cursor.execute(sql)
            
            # adding data to DimFuncao
            for funcao in dataset['Código Função'].unique():
                dataset_aux = dataset[dataset['Código Função'] == funcao]
                
                subfuncao = dataset_aux[dataset_aux['Código Subfunção']].unique()[0]
                acao = dataset_aux[dataset_aux['Código Ação']].unique()[0]
                sql = """INSERT INTO DimFuncao(COD_FUNCAO, COD_SUBFUNCAO, COD_ACAO) VALUES({}, {}, {})""".format(funcao, subfuncao, acao)
                cursor.execute(sql)
            
            # add Data to DimFonte
            for fonte in dataset['Fonte'].unique():
                dataset_aux = dataset[dataset['Código Função'] == fonte]
                
                finalidade = dataset_aux[dataset_aux['Finalidade']].unique()[0]
                sql = """INSERT INTO DimFuncao(FONTE, FINALIDADE) VALUES("{}", "{}")""".format(fonte, finalidade)
                cursor.execute(sql)
            
            #add Data to DimTempo
            for data in dataset['Mês Competência'].unique():
                dia, mes = data.split('/')
                sql = """INSERT INTO DimTempo VALUES("{}", "{}", '28')""".format(dia, mes)
                
        # connection is not autocommit by default. So you must commit to save
        # your changes.
        connection.commit()
    except:
        print(SQL)
    finally:
        connection.close()

In [32]:
oi = """'{}'""".format(int(10))

In [33]:
print(oi)

'10'


In [49]:
#df['Código Função'].unique()
#df['Código Ação'].unique()
df['Código Subfunção'].unique()

array([244], dtype=int64)

In [44]:
for data in df['Mês Competência'].unique():
    print(data.split('/'))

['01', '2017']
['12', '2016']
['11', '2016']


In [48]:
load_data(df)

In [56]:
dataset_aux = df[df['Código Função'] == 8]
dataset_aux.info()             


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 15 columns):
UF                        1000 non-null object
Código SIAFI Município    1000 non-null int64
Nome Município            1000 non-null object
Código Função             1000 non-null int64
Código Subfunção          1000 non-null int64
Código Programa           1000 non-null int64
Código Ação               1000 non-null int64
NIS Favorecido            1000 non-null int64
Nome Favorecido           1000 non-null object
Valor Parcela             1000 non-null float64
Mês Competência           1000 non-null object
Regiao                    1000 non-null object
Estado                    1000 non-null object
Fonte                     1000 non-null object
Finalidade                1000 non-null object
dtypes: float64(1), int64(6), object(8)
memory usage: 125.0+ KB


In [67]:
df.columns

Index(['UF', 'Código SIAFI Município', 'Nome Município', 'Código Função',
       'Código Subfunção', 'Código Programa', 'Código Ação', 'NIS Favorecido',
       'Nome Favorecido', 'Valor Parcela', 'Mês Competência', 'Regiao',
       'Estado', 'Fonte', 'Finalidade'],
      dtype='object')

In [79]:
for col in df.columns:
    print("'{}'".format(col))

'UF'
'Código SIAFI Município'
'Nome Município'
'Código Função'
'Código Subfunção'
'Código Programa'
'Código Ação'
'NIS Favorecido'
'Nome Favorecido'
'Valor Parcela'
'Mês Competência'
'Regiao'
'Estado'
'Fonte'
'Finalidade'


In [82]:
df['UF'][0]

'BA'

In [87]:
dia, mes = df['Mês Competência'][10].split('/')

In [107]:
df[df['Nome Município'] == 'PACATUBA']['Código SIAFI Município'].unique()

array([3197, 1495], dtype=int64)

In [140]:
for cod in df[df['Nome Município'] == 'PACATUBA']['Código SIAFI Município'].unique():
    for city in df[df['Código SIAFI Município'] == cod]:
        print(city.split())

['UF']
['Código', 'SIAFI', 'Município']
['Nome', 'Município']
['Código', 'Função']
['Código', 'Subfunção']
['Código', 'Programa']
['Código', 'Ação']
['NIS', 'Favorecido']
['Nome', 'Favorecido']
['Valor', 'Parcela']
['Mês', 'Competência']
['Regiao']
['Estado']
['Fonte']
['Finalidade']
['UF']
['Código', 'SIAFI', 'Município']
['Nome', 'Município']
['Código', 'Função']
['Código', 'Subfunção']
['Código', 'Programa']
['Código', 'Ação']
['NIS', 'Favorecido']
['Nome', 'Favorecido']
['Valor', 'Parcela']
['Mês', 'Competência']
['Regiao']
['Estado']
['Fonte']
['Finalidade']


In [154]:
print("%.2f" % df['Valor Parcela'][0])

131.00
